<h1>Comparando países, quais gêneros mais populares em cada ano em cada país?

In [1]:
import pandas as pd

total_charts = pd.read_csv('../my_data/charts/total_charts.csv')
total_charts = total_charts[total_charts['country'] != 'global']

total_charts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 242800 entries, 0 to 273599
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   position            242800 non-null  int64  
 1   song_id             242800 non-null  object 
 2   song_name           242662 non-null  object 
 3   artist              242664 non-null  object 
 4   streams             242800 non-null  int64  
 5   last_week_position  211814 non-null  float64
 6   weeks_on_chart      242800 non-null  int64  
 7   peak_position       242800 non-null  int64  
 8   position_status     242800 non-null  object 
 9   year                242800 non-null  int64  
 10  week                242800 non-null  int64  
 11  country             242800 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 24.1+ MB


In [2]:
artists = pd.read_csv('../songs_data/artists/spotify_artists_info_complete_reduced_genres.csv', delimiter='\t')
artists.columns

Index(['artist_id', 'name', 'followers', 'popularity', 'genres', 'image_url'], dtype='object')

In [3]:
total_charts.columns

Index(['position', 'song_id', 'song_name', 'artist', 'streams',
       'last_week_position', 'weeks_on_chart', 'peak_position',
       'position_status', 'year', 'week', 'country'],
      dtype='object')

In [4]:
total_charts_artists = pd.merge(total_charts, artists, left_on='artist', right_on='name')
total_charts_artists.columns

Index(['position', 'song_id', 'song_name', 'artist', 'streams',
       'last_week_position', 'weeks_on_chart', 'peak_position',
       'position_status', 'year', 'week', 'country', 'artist_id', 'name',
       'followers', 'popularity', 'genres', 'image_url'],
      dtype='object')

In [5]:
colunas_desejadas = ['song_id', 'song_name','genres', 'country', 'streams', 'year']
data = total_charts_artists[colunas_desejadas]

In [6]:
from ast import literal_eval

data_analysis = data.copy()
data_analysis['genres'] = data_analysis['genres'].apply(literal_eval)
data_analysis = data_analysis.explode('genres').reset_index(drop=True)
data_analysis
# x = data['genres'][0]
# print(type(x))

,song_id,song_name,genres,country,streams,year
0,5aAx2yezTd8zXrkmtKl66Z,Starboy,r&b,australia,947261,2017
1,5aAx2yezTd8zXrkmtKl66Z,Starboy,pop,australia,947261,2017
2,5GXAXm5YOmYT0kL5jHvYBt,I Feel It Coming,r&b,australia,727010,2017
3,5GXAXm5YOmYT0kL5jHvYBt,I Feel It Coming,pop,australia,727010,2017
4,4F7A0DXBrmUAkp32uenhZt,Party Monster,r&b,australia,218444,2017
...,...,...,...,...,...,...
628236,5FkoSXiJPKTNyYgALRJFhD,ORANGE SODA,trap,united_states,2028636,2019
628237,5FkoSXiJPKTNyYgALRJFhD,ORANGE SODA,hip hop,united_states,2028636,2019
628238,4Xbotg4PCLJw9cDx2dtZLK,Some Things Never Change,other,united_states,1589399,2019
628239,0Q2n5yzl2XRqYbV77tZNYp,GOODMORNINGTOKYO!,rap,united_states,2191423,2019


In [7]:
final_data = data_analysis.copy()
final_data = final_data.groupby(['genres', 'year', 'country']).sum().reset_index()
# final_data.to_csv('q4_data.csv')

In [8]:
import altair as alt

list_genres = final_data['genres'].unique().list()
genres_dropdown = alt.binding_select(
    options=list_genres, 
    name='Gênero Musical: '
)
x_param = alt.selection_single(
    fields=['interval'], 
    bind=input_dropdown
)


grafico = alt.Chart(final_data).mark_bar().encode(
    x=alt.X('X', type='nominal', title='Gênero Musical'),
    Y=alt.Y('streams', type='quantitative'),
).transform_calculate(
    x=f'datum[{x_param.name}]'
)

# add a layer to group bars with respect to countries
alt.layer(bars).facet(
    column='country'
)

AttributeError: 'numpy.ndarray' object has no attribute 'list'